## Viz complémentaire axe 1 : regarder si on observe mieux le déclin des flux commerciaux de La Rochelle en excluant les activités de traite

#### Méthodologie d'acquisition des données de pesée globale adoptée lors du sprint :
* prendre tous les flux toflit avec le flag best_guess_national_partner pour établir les totaux pour toute la france
* prendre tous les flux toflit avec le flag best_guess_region_prodxpart pour établir les totaux par région

#### Ajustements post-sprint : tester sans la traite => comment identifie-ton les activités de traite chez Toflit ?
1. exclure tous les flux avec pour produit "êtres humains" "traite" ??? => on n'a rien de tout ça dans les produits toflit
2. fonctionner avec 'partner' = colonies et aviser en fonction de l'origine du produit ??
=> (je regarde ce tableau qui m'indique que cette technique ne semble pas fonctionner pour les exports : https://docs.google.com/spreadsheets/d/1ChDCPYVhL3E7MgWMXuFQppKiLUPHuaxD48kAF--Tmmo/edit#gid=1433939378)
3. pour les imports : tests à la fin de ce notebook => on n'a pas les origines des produits, et la nature des produits commercés avec les colonies ne semble pas mentionner la traite 

#### approches 1. 2. et 3. pas concluants pour l'instant
==> je dois fonctionner avec des données Navigo pour identifier des activités de traite ?

In [ ]:
from poitousprint import Toflit
from vega import VegaLite
import pandas as pd

toflit_client = Toflit()

In [ ]:
flows = toflit_client.get_flows(best_guess_region_prodxpart='1')

In [ ]:
flows_national = toflit_client.get_flows(best_guess_national_partner='1')

In [ ]:
years_list = [y + 1712 for y in range(120)]
years = {}
# partner_for_imports = set()
# origin_for_imports = set()
# products_imported_from_colonies = set()

for y in years_list:
    years[str(y)] = {
        "year": y,
        "france_total": 0,
        "france_export": 0,
        "france_import": 0,
        
        "la_rochelle_total": 0,
        "la_rochelle_export": 0,
        "la_rochelle_import": 0,
    }


for f in flows_national:
    year = str(f['year'].split('.')[0])
    value = float(f['value']) if f['value'] != '' else 0
    itype = f['export_import'] if f['export_import'] != 'import' else 'Imports'

    detailed_field = 'france_import' if itype == 'Imports' else 'france_export'
    years[year]['france_total'] += value        
    years[year][detailed_field] += value

    
for f in flows:
    year = str(f['year'].split('.')[0])
    value = float(f['value']) if f['value'] != '' else 0
    itype = f['export_import'] if f['export_import'] != 'import' else 'Imports'
    from_larochelle = f['customs_region'] == 'La Rochelle'
    if from_larochelle:
        detailed_field = 'la_rochelle_import' if itype == 'Imports' else 'la_rochelle_export'
        years[year]['la_rochelle_total'] += value        
        years[year][detailed_field] += value
    if itype == 'Imports':
        # partner_for_imports.add(f['partner_simplification'])
        # origin_for_imports.add(f['origin_origin'])
        if f['partner_orthographic'] in ["Sénégal", "Saint-Domingue", "Amérique"]:
            products_imported_from_colonies.add(f['product_simplification'])

part_by_year = []
for year, values in years.items():
    part_by_year.append({
        "year": year,
        "type": "import",
        "portion": values['la_rochelle_import'] / values['france_total'] if  values['france_total'] > 0 else 0
    })
    part_by_year.append({
        "year": year,
        "type": "export",
        "portion": values['la_rochelle_export'] / values['france_total'] if  values['france_total'] > 0 else 0
    })
    """

    part_by_year.append({
        "year": year,
        "type": "reste de la France",
        "portion": 1 - values['la_rochelle_total'] / values['france_total'] if  values['france_total'] > 0 else 0
    })
    """
VegaLite({
    "$schema": "https://vega.github.io/schema/vega-lite/v5.json",
    "title": "Évolution globale de la part des échanges de La Rochelle par rapport à l'ensemble de la France",
    "mark": {"type": "bar", "tooltip": {"content": "data"}},
    "encoding": {
        "x": {
            "field": "year",
            "type": "ordinal", 
            "title": "année"
        },
        "y": {
            "type": "quantitative",
            "field": "portion",
            "title": "Par par rapport au total des échanges en France"
        },
        "color": {
            "type": "nominal",
            "field": "type",
            "title": "Type de flux",
            "scale": {
                "range": ["orange", "blue", "white"]
            }
        }
    }
}, pd.DataFrame([y for y in part_by_year if int(y['year']) > 1717 and int(y['year']) < 1790]))
